# Schnorr digital signatures and verification
## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes his public key $P = dG = (P_x, P_y)$

In [4]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))

ckp.create_keypair(n_keys)["users"]

[{'privateKey': '1562ab67aacd4362c4126ddc5fb02f81348ac648fd41cc855647309191099567',
  'publicKey': '4a6c09fae931499ef8f9e1cf128ad7ce08d862ee428a690444855a541ab16675'},
 {'privateKey': '374d512bf0b676c1d4146c8c4f511cc5ab8f325bbfa0ad39651f49677aaf738f',
  'publicKey': '2191238774dbfad0a24cd7dbc1e8f00b343c225f686ab2ba39ab1b304a9ef20e'},
 {'privateKey': '0bd7c288980346c80fed43475da5692d67d4b297c8a22265f45c963c1bcf9191',
  'publicKey': '0e0a8fc4b6e5521ce502875125bbb2639dfe1945cba6c07a79a40cc6f30c3897'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A calculates $e = h(R || P || M)$
- A calculates $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [8]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =",user[0]["publicKey"])
print("Signature =",sig.hex())

> PublicKey = a1454569bb5a2408251adc19fdf9ec6b0f370496e4af712f6f39aa9291b72575
> Signature = 878248f1b840d7c33115de36ff8dbd91e64af6708a8ae2d97668cafe78da70f2410f5f1b6ef8b7fade8470232c28994acc7dc59c05a61a2bd88e87fb68d324db


## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(\tilde{X}||R||M)$
- Every user computes $s_i = r_i + c\; d_i\; a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

In [9]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig, X = sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

PublicKey = b49f684727a1f3febffed24a5116520754659170d3a6e818ea892659d43aab27
Signature = 9ba7ea569960e33d68829f489fbdd5c169da4d26b04c02b89f1358b9c98cf5f010b2676e9a39fe59eebb635680a8a8386bf37a98430fc6bc7d27d2836e4d2209


## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(\tilde{X}||R_1||...||R_{\nu}||M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(\tilde{X}||R||M)$, then every user computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

In [5]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

M = input("Insert the message to sign:")
M = sl.sha256(M.encode())

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

> PublicKey = cc2890b7095f968683f78c58cf487bff57636d3bfadc5d3a806a4b90718473b5
> Signature = 9e09d67c2bfa93280c212971491133091fee9df2cebd49a7cc580b588657bce675fa213d5c232e77cd6fff392cc5e0be3c5dcaac897872a4b93e90dcf8ef71e3


## Verify Schnorr

- $B$, the receiver, calculates $R_{v} = sG$
- $B$ calculates $e = h(R || P || M)$
- if $R_{v} - R = eP$ the signature is valid  $(P$ is equal to $\tilde{X}$ in the MuSig cases $)$

In [10]:
import schnorr_lib as sl

M = input("Insert the message to verify:")
M = sl.sha256(M.encode())

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is valid for this message and this public key
